In [ ]:
import requests
from bs4 import BeautifulSoup  
import pandas as pd


In [ ]:
website_url = 'https://countrymusichalloffame.org/hall-of-fame/members/'
response = requests.get(website_url)

response.status_code

In [ ]:
print(type(response))
response.content

In [ ]:
country_star = BeautifulSoup(response.content)
print(country_star.title)

In [ ]:
print(country_star)

In [ ]:
members = country_star.find_all('div', attrs = {'class': 'vertical-card_content--title'})
members

In [ ]:
for members in members:
     print(members, end='\n'*2)

artist = []
for i in range(len(members)):
    name = members[i].get_text.replace('\n','')
artist.append(name)
print(artist)
type(members)

In [ ]:
####AMAZON TRIAL:DATACAMP

In [ ]:
no_pages = 1

def get_data(pageNo):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    r = requests.get('https://countrymusichalloffame.org/hall-of-fame/members/'+str(pageNo)+str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)
    
    #Similar to what Holland share with us on artist names
    alls = []
    for d in soup.findAll('div', attrs={'class':'vertical-card'}):
        #print(d)
        name = d.find('div', attrs={'class':'vertical-card_content--title'})
        author = d.find('div', attrs={'class':'Roy Acuff'})
        rating = d.find('div', attrs={'class':'vertical-card_content--copy'})#change to inducted for year
       
    return alls

In [ ]:
results = []
for i in range(1, no_pages+1):
    results.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Book Name','Author','Rating','Customers_Rated', 'Price'])
df.to_csv('CMHOF.csv', index=False, encoding='utf-8')

In [ ]:
df = pd.read_csv("CMHOF.csv")

In [ ]:
df.shape

In [ ]:
df.head(61)